In [1]:
import seaborn as sns
import os
from scipy.stats import kurtosis, skew

from rcv_distribution import *
from MDS_analysis import *
from voting_rules import *
import random
from itertools import permutations
from collections import defaultdict
from collections import Counter

In [2]:
directory = "dataverse_files"
table = pd.read_csv("diff.csv")
for filename in os.listdir(directory):
    try:
        csv = os.path.join(directory, filename)
        ballots, candidates = parse_election_data(csv)

        # Perform the RCV analysis
        test = perform_rcv_analysis(ballots, candidates, n_init = 100, max_itr = 1000, n_runs=1000, metric=False)
        mds_1d_coordinates, mds_2d_coordinates, most_common_order, order_frequencies, candidate_names = test

        # Print the normalized distances between candidates and plot the MDS analysis
        normalized_distances_original = get_distances_normalized(most_common_order, mds_1d_coordinates, candidate_names)

        consistent_ballots, gamma = get_permissive_gamma(ballots, normalized_distances_original)
        table.loc[table["filename"]==filename, "gamma_n_init"] = gamma

        plot_KDE(ballots, normalized_distances_original, filename, ignore=False, save=True)

        freqs = get_frequency(ballots, candidates)
        ignore = [] 
        for c in freqs:
            if freqs[c] < 0.02:
                ignore.append(c)
        if len(ignore) > 0:
            ballots, candidates = parse_election_data(os.path.join("dataverse_files", filename), ignore_values=ignore)
            # Perform the RCV analysis
            test = perform_rcv_analysis(ballots, candidates, n_init = 100, max_itr = 1000, n_runs=1000, metric=False)
            mds_1d_coordinates, mds_2d_coordinates, most_common_order, order_frequencies, candidate_names = test

            # Get the normalized distances between candidates and plot the MDS analysis
            normalized_distances_ignored = get_distances_normalized(most_common_order, mds_1d_coordinates, candidate_names)

            consistent_ballots, gamma = get_permissive_gamma(ballots, normalized_distances_ignored)

            table.loc[table["filename"]==filename, "gamma_ignored"] = gamma

            plot_KDE(ballots, normalized_distances_original, filename, ignore=True, save=True)
    
    except Exception as e:
        # Print the summary of the error
        print(f"An error occurred: {e}")

table.to_csv("diff2.csv")


c:\Users\mahsh\Documents\GitHub\bugs-in-democracy\cleaned files\MDS_analysis.py:162: RuntimeWarning: divide by zero encountered in true_divide
  distance = 1 / np.sqrt(freq_upper_triangle)
c:\Users\mahsh\Documents\GitHub\bugs-in-democracy\cleaned files\MDS_analysis.py:162: RuntimeWarning: divide by zero encountered in true_divide
  distance = 1 / np.sqrt(freq_upper_triangle)
c:\Users\mahsh\Documents\GitHub\bugs-in-democracy\cleaned files\MDS_analysis.py:162: RuntimeWarning: divide by zero encountered in true_divide
  distance = 1 / np.sqrt(freq_upper_triangle)
c:\Users\mahsh\Documents\GitHub\bugs-in-democracy\cleaned files\MDS_analysis.py:162: RuntimeWarning: divide by zero encountered in true_divide
  distance = 1 / np.sqrt(freq_upper_triangle)
c:\Users\mahsh\Documents\GitHub\bugs-in-democracy\cleaned files\MDS_analysis.py:162: RuntimeWarning: divide by zero encountered in true_divide
  distance = 1 / np.sqrt(freq_upper_triangle)
c:\Users\mahsh\Documents\GitHub\bugs-in-democracy\clean

An error occurred: zero-size array to reduction operation minimum which has no identity


c:\Users\mahsh\Documents\GitHub\bugs-in-democracy\cleaned files\MDS_analysis.py:162: RuntimeWarning: divide by zero encountered in true_divide
  distance = 1 / np.sqrt(freq_upper_triangle)
c:\Users\mahsh\Documents\GitHub\bugs-in-democracy\cleaned files\MDS_analysis.py:162: RuntimeWarning: divide by zero encountered in true_divide
  distance = 1 / np.sqrt(freq_upper_triangle)
c:\Users\mahsh\Documents\GitHub\bugs-in-democracy\cleaned files\MDS_analysis.py:162: RuntimeWarning: divide by zero encountered in true_divide
  distance = 1 / np.sqrt(freq_upper_triangle)


An error occurred: zero-size array to reduction operation minimum which has no identity


c:\Users\mahsh\Documents\GitHub\bugs-in-democracy\cleaned files\MDS_analysis.py:162: RuntimeWarning: divide by zero encountered in true_divide
  distance = 1 / np.sqrt(freq_upper_triangle)
c:\Users\mahsh\Documents\GitHub\bugs-in-democracy\cleaned files\MDS_analysis.py:162: RuntimeWarning: divide by zero encountered in true_divide
  distance = 1 / np.sqrt(freq_upper_triangle)
c:\Users\mahsh\Documents\GitHub\bugs-in-democracy\cleaned files\MDS_analysis.py:162: RuntimeWarning: divide by zero encountered in true_divide
  distance = 1 / np.sqrt(freq_upper_triangle)


...

c:\Users\mahsh\Documents\GitHub\bugs-in-democracy\cleaned files\MDS_analysis.py:162: RuntimeWarning: divide by zero encountered in true_divide
  distance = 1 / np.sqrt(freq_upper_triangle)


An error occurred: zero-size array to reduction operation minimum which has no identity


c:\Users\mahsh\Documents\GitHub\bugs-in-democracy\cleaned files\MDS_analysis.py:151: RuntimeWarning: divide by zero encountered in true_divide
  frequencies = counts / mentioned_together
c:\Users\mahsh\Documents\GitHub\bugs-in-democracy\cleaned files\MDS_analysis.py:151: RuntimeWarning: invalid value encountered in true_divide
  frequencies = counts / mentioned_together
c:\Users\mahsh\Documents\GitHub\bugs-in-democracy\cleaned files\MDS_analysis.py:162: RuntimeWarning: divide by zero encountered in true_divide
  distance = 1 / np.sqrt(freq_upper_triangle)


An error occurred: zero-size array to reduction operation minimum which has no identity


c:\Users\mahsh\Documents\GitHub\bugs-in-democracy\cleaned files\MDS_analysis.py:162: RuntimeWarning: divide by zero encountered in true_divide
  distance = 1 / np.sqrt(freq_upper_triangle)
c:\Users\mahsh\Documents\GitHub\bugs-in-democracy\cleaned files\MDS_analysis.py:151: RuntimeWarning: divide by zero encountered in true_divide
  frequencies = counts / mentioned_together
c:\Users\mahsh\Documents\GitHub\bugs-in-democracy\cleaned files\MDS_analysis.py:151: RuntimeWarning: invalid value encountered in true_divide
  frequencies = counts / mentioned_together
c:\Users\mahsh\Documents\GitHub\bugs-in-democracy\cleaned files\MDS_analysis.py:162: RuntimeWarning: divide by zero encountered in true_divide
  distance = 1 / np.sqrt(freq_upper_triangle)
c:\Users\mahsh\Documents\GitHub\bugs-in-democracy\cleaned files\MDS_analysis.py:151: RuntimeWarning: divide by zero encountered in true_divide
  frequencies = counts / mentioned_together
c:\Users\mahsh\Documents\GitHub\bugs-in-democracy\cleaned file